# System Information

Autogenerated info about each system

In [ ]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

import json, pprint

In [ ]:

EXCLUDE=['hostname','BogoMIPS','CPU MHz']
INFO=['os.kernel', 'os.release.PRETTY_NAME', 'chassis.*', 'cpu.*', 'net.*', 'memory.*']

for path in modules.utils.find_run_outputs(root='../../output', test='Sysinfo', ext='.json'):
    with open(path) as f:
        meta = modules.utils.parse_path_metadata(path)
        sysinfo = json.load(f)
        
        print('%s:%s' % (meta['sysname'], meta['partition']))
        for k in sysinfo:
            if k in EXCLUDE:
                continue
            print('== %s ==' % k)
            pprint.pprint(sysinfo[k])
